In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import json
import sys

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
answers = {}

In [7]:
# Some data structures that will be useful

In [8]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [9]:
len(allRatings)

200000

In [10]:
print(allRatings[0])

('u67805239', 'b61372131', 4)


In [11]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [12]:
print(ratingsTrain[0])

('u67805239', 'b61372131', 4)


In [13]:
##################################################
# Rating prediction (CSE258 only)                #
##################################################

In [14]:
### Question 9

In [15]:
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())

In [16]:
ratingMean = sum([d[2] for d in ratingsTrain]) / N

In [17]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [18]:
alwaysPredictMean = [ratingMean for d in ratingsTrain]

In [37]:
labels = [r for _,_,r in ratingsTrain]
print("Always Predict Mean MSE for Train set: {}".format(MSE(alwaysPredictMean, labels)))


Always Predict Mean MSE for Train set: 1.741577477865528


In [36]:
labels = [r for _,_,r in ratingsValid]
print("Always Predict Mean MSE for Valid set: {}".format(MSE(alwaysPredictMean, labels)))

Always Predict Mean MSE for Valid set: 1.6802113179223874


In [126]:
alpha = ratingMean
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
lamb = 1

In [127]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [128]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [222]:
def cost(alpha, userBiases, itemBiases, lamb, dataset):
    #unpack(theta)
    predictions = [prediction(u, b) for u,b,_ in dataset]
    labels = [r for _,_,r in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [130]:
def update_alpha(ratingsTrain, userBiases, itemBiases):
    rsum = sum(r for _, _, r in ratingsTrain)
    userBsum = sum(b for b in userBiases.values())
    itemBsum = sum(i for i in itemBiases.values())
        
    return (rsum - (userBsum + itemBsum)) / N

In [131]:
def update_userBias(u, ratingsPerUser, alpha, itemBiases, lamb):
    items = [i for i, _ in ratingsPerUser[u]]
    rsum = sum(r for _, r in ratingsPerUser[u])
    alphasum = sum(alpha for i in items)
    itemBsum = sum(itemBiases[i] for i in items)
    
    return (rsum - (alphasum + itemBsum)) / (lamb + len(items))
    

In [132]:
def update_itemBias(i, ratingsPerItem, alpha, userBiases, lamb):
    users = [u for u, _ in ratingsPerItem[i]]
    rsum = sum(r for _, r in ratingsPerItem[i])
    alphasum = sum(alpha for u in users)
    userBsum = sum(userBiases[u] for u in users)
    
    return (rsum - (alphasum + userBsum)) / (lamb + len(users))

In [133]:
def derivative(alpha, userBiases, itemBiases, labels, lamb):
    #unpack(theta)
    delta = sys.float_info.max
    threshold = 0.01
    maxItrs = 1e4
    cnt = 0
    #oldCost = cost(alpha, userBiases, itemBiases, lamb)
    preds = [prediction(u, b) for u, b, _ in ratingsTrain]
    oldMSE = MSE(preds, labels)
    
    while delta > threshold and cnt < maxItrs:      
        alpha = update_alpha(ratingsTrain, userBiases, itemBiases)
        
        for u,b in userBiases.items():
            newUserBias = update_userBias(u, ratingsPerUser, alpha, itemBiases, lamb)
            userBiases[u] = newUserBias
        
        for i,b in itemBiases.items():
            newItemBias = update_itemBias(i, ratingsPerItem, alpha, userBiases, lamb)
            itemBiases[i] = newItemBias
        
        #newCost = cost(alpha, userBiases, itemBiases, lamb)
        #delta = newCost - oldCost
        
        preds = [prediction(u, b) for u, b, _ in ratingsTrain]
        newMSE = MSE(preds, labels)
        delta = newMSE - oldMSE
        print(delta)
        print(newMSE)
        cnt = cnt + 1

In [134]:
labels = [r for _, _, r in ratingsTrain]
derivative(alpha, userBiases, itemBiases, labels, lamb)

preds = [prediction(u, b) for u, b, _ in ratingsTrain]
print("Mean MSE for Train set: {}".format(MSE(preds, labels)))

-0.6688620074174276
1.0727154704481003
Mean MSE for Train set: 1.0727154704481003


In [135]:
preds = [prediction(u, b) for u, b, _ in ratingsValid]
labels = [r for _, _, r in ratingsValid]
validMSE = MSE(preds, labels)
print("Mean MSE for Valid set: {}".format(MSE(preds, labels)))

Mean MSE for Valid set: 1.4406701055112554


In [136]:
print(validMSE)

1.4406701055112554


In [42]:
answers['Q9'] = validMSE

In [43]:
assertFloat(answers['Q9'])

In [ ]:
### Question 10

In [137]:
print(max(userBiases))
print(max(userBiases, key=userBiases.get))
print(userBiases['u99994977'])
print(userBiases['u19874911'])

u99994977
u55363066
0.6117286549707607
0.9848960526315791


In [138]:
print(min(userBiases))
print(min(userBiases, key=userBiases.get))
print(userBiases['u00000114'])
print(userBiases['u56426501'])

u00000114
u56426501
-0.6009546052631576
-3.5393330526315783


In [139]:
maxUser = max(userBiases, key=userBiases.get)
maxBeta = userBiases[maxUser]

minUser = min(userBiases, key=userBiases.get)
minBeta = userBiases[minUser]

print(maxUser)
print(minUser)
print(maxBeta)
print(minBeta)

u55363066
u56426501
1.2721574013157904
-3.5393330526315783


In [140]:
answers['Q10'] = [maxUser, minUser, maxBeta, minBeta]

In [141]:
assert [type(x) for x in answers['Q10']] == [str, str, float, float]

In [142]:
### Question 11

In [267]:
alpha = ratingMean
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
lamb = 4.5

In [268]:
labels = [r for _, _, r in ratingsTrain]
derivative(alpha, userBiases, itemBiases, labels, lamb)

preds = [prediction(u, b) for u, b, _ in ratingsTrain]
print("Mean MSE for Train set: {}".format(MSE(preds, labels)))

-0.5896255908129304
1.1519518870525975
Mean MSE for Train set: 1.1519518870525975


In [269]:
preds = [prediction(u, b) for u, b, _ in ratingsValid]
labels = [r for _, _, r in ratingsValid]
validMSE = MSE(preds, labels)
print("Mean MSE for Valid set: {}".format(MSE(preds, labels)))

Mean MSE for Valid set: 1.38642208414719


In [270]:
c = cost(alpha, userBiases, itemBiases, lamb, ratingsValid)
print(c)

MSE = 1.38642208414719
26489.068856835594


In [271]:
answers['Q11'] = (lamb, validMSE)

In [272]:
assertFloat(answers['Q11'][0])
assertFloat(answers['Q11'][1])

In [273]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    p = prediction(u, b)
    predictions.write("{},{},{}\n".format(u,b,p))
    
predictions.close()

In [ ]:
##################################################
# Read prediction                                #
##################################################

In [58]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [59]:
def read_pred(b, return1):
    if b in return1:
        return 1
    else:
        return 0

In [60]:
### Question 1

In [61]:
def sample_read(u, allBooks):
    readBooks = set()
    
    for v, b,_ in allRatings:
        if u == v:
            readBooks.add(b)
    
    return readBooks

def sample_unread(readBooks, allBooks):
    unreadBooks = set()

    for b in allBooks:
        if b not in readBooks:
            unreadBooks.add(b)

    return unreadBooks
    

In [62]:
allBooks = set()

for _,b,_ in allRatings:
    allBooks.add(b)

In [63]:
print(len(allBooks))

6688


In [64]:
negSamples = []
posSamples = []
newValid = []

for u,b,r in ratingsValid:
    readBooks = sample_read(u, allRatings)
    randReadBook = random.choice(tuple(readBooks))
    print("randReadbook : {}".format(randReadBook))
    posSamples.append((u, randReadBook, 1))
    newValid.append((u, randReadBook, 1))
    
    unreadBooks = sample_unread(readBooks, allBooks)
    if len(unreadBooks) != 0:
        randUnreadBook = random.choice(tuple(unreadBooks))
        print("randUnreadbook : {}".format(randUnreadBook))
        negSamples.append((u, randUnreadBook, 0))
        newValid.append((u, randUnreadBook, 0))    

randReadbook : b42977080
randUnreadbook : b08899763
randReadbook : b42058394
randUnreadbook : b68936917
randReadbook : b31362596
randUnreadbook : b62474102
randReadbook : b67956548
randUnreadbook : b49401737
randReadbook : b78457511
randUnreadbook : b13395935
randReadbook : b23369233
randUnreadbook : b60481592
randReadbook : b85680306
randUnreadbook : b79091567
randReadbook : b98413533
randUnreadbook : b77915857
randReadbook : b30188578
randUnreadbook : b92726188
randReadbook : b76107444
randUnreadbook : b24571577
randReadbook : b21557753
randUnreadbook : b68763841
randReadbook : b07855606
randUnreadbook : b72415274
randReadbook : b14890131
randUnreadbook : b34792889
randReadbook : b71450314
randUnreadbook : b34984766
randReadbook : b21498435
randUnreadbook : b88261609
randReadbook : b71294015
randUnreadbook : b09109148
randReadbook : b58230280
randUnreadbook : b00524913
randReadbook : b79253970
randUnreadbook : b24802633
randReadbook : b69647795
randUnreadbook : b43038067
randReadbook

randReadbook : b82853645
randUnreadbook : b35228639
randReadbook : b46024181
randUnreadbook : b30263601
randReadbook : b79928088
randUnreadbook : b23536304
randReadbook : b08545824
randUnreadbook : b94465304
randReadbook : b89251055
randUnreadbook : b14113125
randReadbook : b76329691
randUnreadbook : b25779761
randReadbook : b41566519
randUnreadbook : b71640551
randReadbook : b98863746
randUnreadbook : b21699056
randReadbook : b68588879
randUnreadbook : b63652115
randReadbook : b36185570
randUnreadbook : b46873727
randReadbook : b17745126
randUnreadbook : b56787707
randReadbook : b17741315
randUnreadbook : b04391367
randReadbook : b90302136
randUnreadbook : b95021405
randReadbook : b35526856
randUnreadbook : b38877642
randReadbook : b98954571
randUnreadbook : b36766414
randReadbook : b15268547
randUnreadbook : b50733255
randReadbook : b14888446
randUnreadbook : b94124672
randReadbook : b45324974
randUnreadbook : b24809015
randReadbook : b39916665
randUnreadbook : b47960551
randReadbook

randReadbook : b84770627
randUnreadbook : b33849296
randReadbook : b19867465
randUnreadbook : b87934970
randReadbook : b06163986
randUnreadbook : b72545062
randReadbook : b02510280
randUnreadbook : b63640911
randReadbook : b79815294
randUnreadbook : b42348056
randReadbook : b15869276
randUnreadbook : b55670266
randReadbook : b52580354
randUnreadbook : b83548765
randReadbook : b18610759
randUnreadbook : b80954905
randReadbook : b31362596
randUnreadbook : b10490640
randReadbook : b95816463
randUnreadbook : b04575815
randReadbook : b76691430
randUnreadbook : b76793631
randReadbook : b34755064
randUnreadbook : b61139235
randReadbook : b68986624
randUnreadbook : b95130898
randReadbook : b52580354
randUnreadbook : b31919542
randReadbook : b49444033
randUnreadbook : b28968477
randReadbook : b29505914
randUnreadbook : b49324643
randReadbook : b08180481
randUnreadbook : b63176169
randReadbook : b37657413
randUnreadbook : b45236603
randReadbook : b29738200
randUnreadbook : b06138796
randReadbook

randReadbook : b63603925
randUnreadbook : b85462814
randReadbook : b51286612
randUnreadbook : b15576725
randReadbook : b10949180
randUnreadbook : b83778249
randReadbook : b26385014
randUnreadbook : b07815897
randReadbook : b76473315
randUnreadbook : b70929698
randReadbook : b95835140
randUnreadbook : b34146432
randReadbook : b36766414
randUnreadbook : b94847436
randReadbook : b49553867
randUnreadbook : b50275880
randReadbook : b51612619
randUnreadbook : b40715352
randReadbook : b02624512
randUnreadbook : b03961467
randReadbook : b45033558
randUnreadbook : b64524405
randReadbook : b22016281
randUnreadbook : b14574218
randReadbook : b34681163
randUnreadbook : b44991539
randReadbook : b26385014
randUnreadbook : b96855617
randReadbook : b87049222
randUnreadbook : b19623908
randReadbook : b55226287
randUnreadbook : b70760387
randReadbook : b22652791
randUnreadbook : b83644107
randReadbook : b64937687
randUnreadbook : b01338628
randReadbook : b57648353
randUnreadbook : b24116940
randReadbook

randReadbook : b78008834
randUnreadbook : b54729838
randReadbook : b74418157
randUnreadbook : b05356235
randReadbook : b74215590
randUnreadbook : b23970048
randReadbook : b64297339
randUnreadbook : b30801301
randReadbook : b68456479
randUnreadbook : b51927662
randReadbook : b95056678
randUnreadbook : b96244732
randReadbook : b86788370
randUnreadbook : b78330112
randReadbook : b17952463
randUnreadbook : b58751002
randReadbook : b61372131
randUnreadbook : b83439687
randReadbook : b97897718
randUnreadbook : b70044367
randReadbook : b45954724
randUnreadbook : b31285004
randReadbook : b29802159
randUnreadbook : b99519260
randReadbook : b85316838
randUnreadbook : b03399757
randReadbook : b57648353
randUnreadbook : b61005749
randReadbook : b43914262
randUnreadbook : b35638466
randReadbook : b61677779
randUnreadbook : b54859884
randReadbook : b43576258
randUnreadbook : b16428538
randReadbook : b40477781
randUnreadbook : b34090614
randReadbook : b05376454
randUnreadbook : b09037695
randReadbook

...

randReadbook : b45445418
randUnreadbook : b95163960
randReadbook : b88978481
randUnreadbook : b89512519
randReadbook : b57254246
randUnreadbook : b35855543
randReadbook : b18469261
randUnreadbook : b16823165
randReadbook : b80160832
randUnreadbook : b67380674
randReadbook : b78180005
randUnreadbook : b89202972
randReadbook : b18610759
randUnreadbook : b20260779
randReadbook : b26940296
randUnreadbook : b34496636
randReadbook : b90190217
randUnreadbook : b50425922
randReadbook : b43479570
randUnreadbook : b06319379
randReadbook : b36282154
randUnreadbook : b03659745
randReadbook : b82219728
randUnreadbook : b92133639
randReadbook : b80618902
randUnreadbook : b09958531
randReadbook : b98545244
randUnreadbook : b96677697
randReadbook : b21674268
randUnreadbook : b91302109
randReadbook : b90695144
randUnreadbook : b45157835
randReadbook : b28615151
randUnreadbook : b80730701
randReadbook : b43633278
randUnreadbook : b31582404
randReadbook : b81697368
randUnreadbook : b91306932
randReadbook

randReadbook : b15854967
randUnreadbook : b41394080
randReadbook : b69294942
randUnreadbook : b01714936
randReadbook : b16823165
randUnreadbook : b25670333
randReadbook : b55512267
randUnreadbook : b85321421
randReadbook : b78008834
randUnreadbook : b98919800
randReadbook : b41804017
randUnreadbook : b24232915
randReadbook : b95816463
randUnreadbook : b09133711
randReadbook : b54369491
randUnreadbook : b02793254
randReadbook : b58230280
randUnreadbook : b55619369
randReadbook : b34964632
randUnreadbook : b80148153
randReadbook : b63603925
randUnreadbook : b91143227
randReadbook : b36795461
randUnreadbook : b38992011
randReadbook : b16024414
randUnreadbook : b82679092
randReadbook : b67550881
randUnreadbook : b03441179
randReadbook : b54342830
randUnreadbook : b95009595
randReadbook : b38122928
randUnreadbook : b00306483
randReadbook : b13061166
randUnreadbook : b14024522
randReadbook : b02984754
randUnreadbook : b05459898
randReadbook : b21633717
randUnreadbook : b78288340
randReadbook

randReadbook : b61863874
randUnreadbook : b80203303
randReadbook : b00967305
randUnreadbook : b31687952
randReadbook : b42242454
randUnreadbook : b23798629
randReadbook : b05280569
randUnreadbook : b54721829
randReadbook : b58973293
randUnreadbook : b15453587
randReadbook : b46891479
randUnreadbook : b64553144
randReadbook : b34827091
randUnreadbook : b55148065
randReadbook : b71271368
randUnreadbook : b77333717
randReadbook : b13222925
randUnreadbook : b24593494
randReadbook : b52503900
randUnreadbook : b56373143
randReadbook : b91712126
randUnreadbook : b57879270
randReadbook : b31821160
randUnreadbook : b56180380
randReadbook : b12601599
randUnreadbook : b70841152
randReadbook : b37364308
randUnreadbook : b24247877
randReadbook : b70912031
randUnreadbook : b32342963
randReadbook : b22409558
randUnreadbook : b71185712
randReadbook : b72304066
randUnreadbook : b62349347
randReadbook : b56879260
randUnreadbook : b59427381
randReadbook : b15722276
randUnreadbook : b04149119
randReadbook

randReadbook : b90639021
randUnreadbook : b87060879
randReadbook : b35067156
randUnreadbook : b00793465
randReadbook : b80730701
randUnreadbook : b34149514
randReadbook : b99145398
randUnreadbook : b83638329
randReadbook : b33860867
randUnreadbook : b40962529
randReadbook : b16390846
randUnreadbook : b59624169
randReadbook : b08004121
randUnreadbook : b56741431
randReadbook : b20755858
randUnreadbook : b96097640
randReadbook : b48544437
randUnreadbook : b90491722
randReadbook : b75852622
randUnreadbook : b23711476
randReadbook : b78457511
randUnreadbook : b41451971
randReadbook : b05853556
randUnreadbook : b58075017
randReadbook : b05871767
randUnreadbook : b23029053
randReadbook : b92598217
randUnreadbook : b23798629
randReadbook : b80968466
randUnreadbook : b14343904
randReadbook : b76329691
randUnreadbook : b40702452
randReadbook : b29576446
randUnreadbook : b73997973
randReadbook : b87138227
randUnreadbook : b13958434
randReadbook : b00054082
randUnreadbook : b98780773
randReadbook

randReadbook : b08004121
randUnreadbook : b81166341
randReadbook : b69989437
randUnreadbook : b61790818
randReadbook : b06167371
randUnreadbook : b94527972
randReadbook : b76040943
randUnreadbook : b76147424
randReadbook : b92969085
randUnreadbook : b58956630
randReadbook : b03458965
randUnreadbook : b47831110
randReadbook : b73481965
randUnreadbook : b44465294
randReadbook : b08004121
randUnreadbook : b44136369
randReadbook : b49458902
randUnreadbook : b62841431
randReadbook : b35202318
randUnreadbook : b28518292
randReadbook : b67584295
randUnreadbook : b76264408
randReadbook : b80625075
randUnreadbook : b98761363
randReadbook : b08473327
randUnreadbook : b97582573
randReadbook : b39837030
randUnreadbook : b93780977
randReadbook : b29505914
randUnreadbook : b04709328
randReadbook : b10161001
randUnreadbook : b48912946
randReadbook : b00943731
randUnreadbook : b08385011
randReadbook : b64538575
randUnreadbook : b02510280
randReadbook : b18413581
randUnreadbook : b55152063
randReadbook

In [65]:
print(len(posSamples))
print(len(negSamples))
print(len(newValid))

10000
10000
20000


In [66]:
with open('posSamples.json', 'w') as f:
    json.dump(posSamples, f)

In [67]:
with open('negSamples.json', 'w') as f:
    json.dump(negSamples, f)

In [68]:
with open('newValid.json', 'w') as f:
    json.dump(newValid, f)

In [69]:
with open('posSamples.json', 'r') as f:
    posSamples = [tuple(x) for x in json.load(f)]

In [70]:
with open('negSamples.json', 'r') as f:
    negSamples = [tuple(x) for x in json.load(f)]

In [71]:
with open('newValid.json', 'r') as f:
    newValid = [tuple(x) for x in json.load(f)]

In [72]:
correct = 0

for _, b, l in newValid:
    pred = read_pred(b, return1)
    if pred == l:
        correct = correct + 1

acc1 = correct / len(newValid)

In [73]:
print(acc1)

0.7137


In [74]:
answers['Q1'] = acc1

In [75]:
assertFloat(answers['Q1'])

In [76]:
### Question 2

In [77]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
threshold = 1/1.5
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead * threshold: break

In [78]:
correct = 0

for _, b, l in newValid:
    pred = read_pred(b, return1)
    if pred == l:
        correct = correct + 1

acc2 = correct / len(newValid)

In [79]:
print(threshold)
print(acc2)

0.6666666666666666
0.7551


In [80]:
answers['Q2'] = [threshold, acc2]

In [81]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [82]:
### Question 3/4

In [83]:
usersPerBookTrain = defaultdict(set)

for u, b, _ in ratingsTrain:
    usersPerBookTrain[b].add(u)

In [84]:
booksPerUserTrain = defaultdict(set)

for u, b, _ in ratingsTrain:
    booksPerUserTrain[u].add(b)

In [85]:
booksPerUserValid = defaultdict(set)

for u, b, _ in ratingsValid:
    booksPerUserValid[u].add(b)

In [86]:
def Jaccard(s1, s2):
    a = len(s1.intersection(s2))
    b = len(s1.union(s2))
    return 0 if b == 0 else a / b

In [87]:
def mostSimilar(b, booksTrain, usersPerBookTrain):
    sims = []
    users1 = usersPerBookTrain[b]
    
    for b2 in booksTrain:
        if b == b2:
            continue
        else:
            users2 = usersPerBookTrain[b2]
            sim = Jaccard(users1, users2)
            sims.append((sim, b2))
    
    sims.sort(reverse=True)
    
    if len(sims) == 0:
        return None
    
    return sims[0]

In [88]:

def read_pred(u, b, booksPerUserTrain, usersPerBookTrain):
    sims = []
    booksTrain = booksPerUserTrain[u]
    mostSim = mostSimilar(b, booksTrain, usersPerBookTrain)
    threshold = 0.0009
    
    if mostSim is None:
        return 0
    
    sims.append((mostSim[0], b))
    if mostSim[0] > threshold:
        return 1
    else:
        return 0

In [89]:
correct = 0

for u, b, l in newValid :
    if read_pred(u, b, booksPerUserTrain, usersPerBookTrain) == l:
        correct = correct + 1

acc3 = correct / len(newValid)

In [90]:
print(acc3)

0.76


In [91]:
answers['Q3'] = acc3

In [92]:
def read_pred_popularity(b, return1):
    if b in return1:
        return 1
    else:
        return 0

In [93]:
def read_pred_similarity(u, b, booksPerUserTrain, usersPerBookTrain):
    sims = []
    booksTrain = booksPerUserTrain[u]
    mostSim = mostSimilar(b, booksTrain, usersPerBookTrain)
    threshold = 0.0009
    
    if mostSim is None:
        return 0

    sims.append((mostSim[0], b))

    if mostSim[0] > threshold:
        return 1
    else:
        return 0

In [94]:
correct = 0
pred = 0

for u, b, l in newValid :
    predPop = read_pred_popularity(b, return1)
    predSim = read_pred_similarity(u, b,booksPerUserTrain, usersPerBookTrain)
        
    if  predPop == predSim :
        pred = predPop
    else:
        pred = random.choice([predPop, predSim])
    
    if pred == l:
        correct = correct + 1

acc4 = correct / len(newValid)

In [95]:
print(acc4)

0.75955


In [96]:
answers['Q4'] = acc4

In [97]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [98]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)
    predPop = read_pred_popularity(b, return1)
    predSim = read_pred_similarity(u, b, booksPerUserTrain, usersPerBookTrain)
    
    if predPop == predSim:
        p = predPop
    else:
        p = random.choice([predPop, predSim])
    
    predictions.write("{},{},{}\n".format(u,b,p))
predictions.close()

In [99]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [100]:
assert type(answers['Q5']) == str

In [101]:
##################################################
# Category prediction (CSE158 only)              #
##################################################

In [102]:
### Question 6

In [103]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

FileNotFoundError: [Errno 2] No such file or directory: 'train_Category.json.gz'

In [ ]:
data[0]

In [ ]:
answers['Q6'] = counts[:10]

In [ ]:
assert [type(x[0]) for x in answers['Q6']] == [int]*10
assert [type(x[1]) for x in answers['Q6']] == [str]*10

In [ ]:
### Question 7

In [ ]:
Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [ ]:
answers['Q7'] = acc7

In [ ]:
assertFloat(answers['Q7'])

In [ ]:
### Question 8

In [ ]:
answers['Q8'] = acc8

In [ ]:
assertFloat(answers['Q8'])

In [ ]:
# Run on test set

In [ ]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)

predictions.close()

In [204]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()